In [ ]:
# import jax, jax.numpy as jnp
# import math

# from ex_operations import exchange_gate_nqubits
# from ex_operations import exchange_generators

# from gymnax.environments import environment, spaces
# from flax import struct
# import jax, jax.numpy as jnp
# from functools import reduce
# from typing import Tuple
# from fw_target import U_circuit                                



In [1]:
# ------------------------------------------------------------
# 1.  Imports & environment
# ------------------------------------------------------------
import math, jax, jax.numpy as jnp
from exchange_cnot_env import ExchangeCNOTEnv, EnvParams, NEIGHBORS   # your env file
from fw_target          import U_circuit as TARGET_FULL               # (64,64)
from ex_operations      import exchange_gate_nqubits                  # only for sanity
# helper from your env file
from exchange_cnot_env  import _fidelity                              # Frobenius-norm fidelity
from exchange_cnot_env  import _block, LOGICAL                        # (9,9) block indices

## Environment

In [ ]:
# # ----------------------------------------------------------------------
# # 1.  Static problem data
# # ----------------------------------------------------------------------
# N_PHYS = 6
# NEIGHBORS = jnp.array([[0, 1],
#                        [1, 2],
#                        [2, 3],
#                        [3, 4],
#                        [4, 5]], dtype=jnp.int32)   # 5 nearest pairs

# # logical sub-space indices (0-based) ─ rows/cols 
# LOGICAL = jnp.array([9,10,12,17,18,20,33,34,36], dtype=jnp.int32)

# # ----------  target unitary  -----------------------------------------
# # IMPORT YOUR 64×64 TARGET HERE
# from fw_target import U_circuit as TARGET_FULL           # (64,64) complex64
# TARGET_BLOCK = TARGET_FULL[LOGICAL][:, LOGICAL]          # (9,9)
# H_BASE = jnp.stack(
#     [exchange_generators(N_PHYS, int(i), int(j)) for i, j in NEIGHBORS.tolist()],
#     axis=0,                                             # shape (5, 64, 64)
# )


# # ----------------------------------------------------------------------
# # 2.  Small helpers (JAX-friendly, no global state)
# # ----------------------------------------------------------------------
# def _block(U):                               # 9×9 logical block
#     return U[LOGICAL][:, LOGICAL]


# def _fidelity(A: jnp.ndarray, B: jnp.ndarray) -> jnp.float32:
#     """|⟨A,B⟩| / (‖A‖‖B‖)  ϵ [0,1]"""
#     inner = jnp.vdot(A, B)
#     return jnp.abs(inner) / (jnp.linalg.norm(A) * jnp.linalg.norm(B))


# SEL = LOGICAL                                 # for the 27-element mask
# ROWS, COLS = jnp.meshgrid(SEL, SEL, indexing="ij")
# MASK27 = (TARGET_FULL[ROWS, COLS] != 0)        # (9,9) boolean


# # ----------------------------------------------------------------------
# # 3.  Environment data-classes
# # ----------------------------------------------------------------------
# @struct.dataclass(frozen=True)
# class EnvState:
#     U:        jnp.ndarray
#     step:     jnp.int32
#     fid64:    jnp.float32     # last-computed full-matrix fidelity
#     fid9:     jnp.float32     # last-computed block fidelity



# @struct.dataclass
# class EnvParams:
#     max_depth: int = 18
#     dense_obs: bool = False
#     obs_mode: str = "block"   # "block" | "full" | "both"

# # ----------------------------------------------------------------------
# # 4.  Reward function (implements the 13 rules)
# # ----------------------------------------------------------------------
# def reward_fn(U_now, U_prev, step):
#     """
#         Implements the 10 bullet-points exactly (vectorised & JIT-safe).
#         Returns  (reward, done)
#         1. For each step if the fidelity of the 64x64 is increased , +1 to the reward.
#         2. For each step if the fidelity of the 64x64 is increased by .1, +5 reward.
#         3. For each step if the fidelity of the 64x64 is increased by .2, +8 reward.
#         4. For each step if the fidelity of the 9x9 is increased, +.5 to the reward.
#         5. For each step if the fidelity of the 9x9 is increased by .1, +2 reward.
#         6. For each step if the fidelity of the 9x9 is increased by .2, +3 reward.
#         7. If the matrix matched 27 elements of the target in the following rows and columns [9,10,12,17,18,20,33,34,36], then +30 reward.
#         8. For each step if the fidelity with the target matrix is reduced compared to the previous step, then -1 reward.
#         9. For each step if the fidelity with the target matrix is reduced by .1 compared to the previous step, then -2 reward.
#         10. For each step if the fidelity with the target matrix is reduced by .2 compared to the previous step, then -3 reward.
#         11. For each step after the 10th step, step penalty of -1.
#         12. If the matrix matches the entire 64d rows and columns of  the target matrix in the following indices [9,10,12,17,18,20,33,34,36], which would be 9*64*2 elements, +80 reward and exit.
#         13. If the fidelity is greater than .99, +100 reward and exit.
#     """
#     f64_now = _fidelity(U_now, TARGET_FULL)
#     f9_now = _fidelity(_block(U_now), TARGET_BLOCK)
#     f64_prev = _fidelity(U_prev, TARGET_FULL)
#     f9_prev = _fidelity(_block(U_prev), TARGET_BLOCK)

#     df64 = f64_now - f64_prev
#     df9 = f9_now - f9_prev

#     r = 0.0
#     # 1-3
#     r += jax.lax.select(df64 > 0, 1.0, 0.0)
#     r += jax.lax.select(df64 >= 0.10, 5.0, 0.0)
#     r += jax.lax.select(df64 >= 0.20, 8.0, 0.0)
#     # 4-6
#     r += jax.lax.select(df9 > 0, 0.5, 0.0)
#     r += jax.lax.select(df9 >= 0.10, 2.0, 0.0)
#     r += jax.lax.select(df9 >= 0.20, 3.0, 0.0)
#     # 8-10 penalties
#     r -= jax.lax.select(df64 < 0, 1.0, 0.0)
#     r -= jax.lax.select(df64 <= -0.10, 2.0, 0.0)
#     r -= jax.lax.select(df64 <= -0.20, 3.0, 0.0)
#     # 11 step penalty
#     r -= 0.25* step if step > 10 else 0.0

#     # 7: 27-element exact match
#     match27 = jnp.all(
#         jnp.isclose(U_now[ROWS, COLS][MASK27],
#                     TARGET_FULL[ROWS, COLS][MASK27],
#                     atol=1e-6)
#     )
#     r += jax.lax.select(match27, 30.0, 0.0)

#     # 12: all 9×64×2 real/imag entries of those rows identical
#     full_match = jnp.all(jnp.isclose(U_now[SEL], TARGET_FULL[SEL], atol=1e-6))
#     done_by_match = full_match
#     r += jax.lax.select(done_by_match, 80.0, 0.0)

#     # 13: fidelity ≥ .99
#     done_by_fidelity = f64_now >= 0.99
#     r += jax.lax.select(done_by_fidelity, 100.0, 0.0)

#     done = jnp.logical_or(done_by_match, done_by_fidelity)
#     return r.astype(jnp.float32), done, f64_now.astype(jnp.float32), f9_now.astype(jnp.float32)


# # ----------------------------------------------------------------------
# # 5.  The Gymnax environment
# # ----------------------------------------------------------------------
# class ExchangeCNOTEnv(environment.Environment):

#     # ---- constructor -------------------------------------------------- #
#     def __init__(self, params: EnvParams = EnvParams()):
#         self._params = params
#         super().__init__()
        
#         # sub-spaces reused by observation_space / action_space
#         self._pair_space  = spaces.Discrete(NEIGHBORS.shape[0])             # 5
#         self._angle_space = spaces.Box(
#             low  = -jnp.ones(()),        # lower bound  scalar
#             high =  jnp.ones(()),        # upper bound  scalar
#             shape= (),                   # *scalar* continuous parameter p
#             dtype=jnp.float32,
#         )


#     # ---- spaces ------------------------------------------------------- #
#     # read-only accessor that mirrors the Gymnax interface
#     @property
#     def default_params(self) -> EnvParams:
#         return self._params

#     @property
#     def observation_space(self):
#         mode = self.default_params.obs_mode
#         if mode == "block":
#             dim = 9 * 9 * 2 + 1
#         elif mode == "full":
#             dim = 64 * 64 * 2 + 1
#         elif mode == "both":
#             dim = 64 * 64 * 2 + 9 * 9 * 2 + 1
#         else:
#             raise ValueError(f"Unknown obs_mode {mode}")
#         high = jnp.ones(dim, dtype=jnp.float32)
#         return spaces.Box(-high, high)


#     @property
#     def action_space(self):
#         """Tuple (pair_id, p)."""
#         return spaces.Tuple((self._pair_space, self._angle_space))

#     # ---- helpers ------------------------------------------------------ #
#     @staticmethod
#     @jax.jit
#     def _apply(U, pair_id, p):
#         H = H_BASE[pair_id]                                # gather (JIT-safe)
#         U_gate = jax.scipy.linalg.expm(-1j * jnp.pi * p * H)
#         return U_gate @ U
#     def _vec(self, U, step):
#         mode = self.default_params.obs_mode
#         parts = []
#         if mode in ("full", "both"):
#             parts += [U.real.flatten(), U.imag.flatten()]
#         if mode in ("block", "both"):
#             blk = _block(U)
#             parts += [blk.real.flatten(), blk.imag.flatten()]
#         parts += [jnp.array([step], dtype=jnp.float32)]
#         return jnp.concatenate(parts).astype(jnp.float32)

#     # ---- reset -------------------------------------------------------- #
#     def reset(self, key: jax.random.PRNGKey,
#               params: EnvParams | None = None) -> Tuple[EnvState, jnp.ndarray]:
#         p = self.default_params if params is None else params
#         U0 = jnp.eye(2 ** N_PHYS, dtype=jnp.complex64)
#         U0 = jnp.eye(2 ** N_PHYS, dtype=jnp.complex64)
#         phase = jnp.vdot(U0, TARGET_FULL)/jnp.abs(jnp.vdot(U0, TARGET_FULL))
#         fid64_0 = _fidelity(U0*phase.conj(), TARGET_FULL)
#         fid9_0 = _fidelity(_block(U0), TARGET_BLOCK)
#         obs0 = self._vec(U0, 0)
#         return EnvState(U0, 0, fid64_0, fid9_0), obs0

#     # ---- step --------------------------------------------------------- #
#     def step(self, key: jax.random.PRNGKey, state: EnvState, action,
#              params: EnvParams | None = None):
#         """action = (pair_id, array([p]))"""
#         p_cfg = self.default_params if params is None else params

#         pair_id, p_val = action               # p_val is length-1 array
#         p_scalar = jnp.squeeze(p_val)            

#         U_next = self._apply(state.U, pair_id, p_scalar)
#         step_no = state.step + 1

#         # reward & termination
#         r, done_intrinsic, f64, f9 = reward_fn(U_next, state.U, step_no)
#         done = jnp.logical_or(done_intrinsic, step_no == p_cfg.max_depth)

#         obs = self._vec(U_next, step_no)
#         new_state = EnvState(U_next, step_no, f64, f9)
#         return new_state, obs, r, done, {}

# # ----------------------------------------------------------------------
# # 6.  Smoke-test
# # ----------------------------------------------------------------------
# if __name__ == "__main__":
#     env = ExchangeCNOTEnv(EnvParams(max_depth=18, obs_mode="both"))
#     key = jax.random.PRNGKey(0)

#     state, obs = env.reset(key)
#     print("obs[0:5] =", obs[:5], "done? ->", False)

#     # random action: choose pair 2, p = 0.3
#     action = (2, jnp.array([0.3], dtype=jnp.float32))
#     state, obs, r, done, _ = env.step(key, state, action)
#     print("reward =", float(r), "done? ->", bool(done))

obs[0:5] = [1. 0. 0. 0. 0.] done? -> False
reward = -1.0 done? -> False


## Unit testing

In [3]:


# ------------------------------------------------------------
# 2.  Fong-Wandzura 22-pulse specification (0-based qubit ids)
# ------------------------------------------------------------
p1 = math.acos(-1 / math.sqrt(3)) / math.pi     # ≈ 0.696…
p2 = math.asin( 1 /             3) / math.pi    # ≈ 0.108…

gate_specs = [
    ( 1+p1,  [3,4] ),
    # ( p1,    [3,4] ),
    ( p2,    [4,5] ),
    ( 0.5,   [2,3] ),
    ( 1.0,   [3,4] ),
    (-0.5,   [2,3] ),
    (-0.5,   [4,5] ),
    ( 1.0,   [1,2] ),
    (-0.5,   [3,4] ),
    (-0.5,   [2,3] ),
    ( 1.0,   [4,5] ),
    (-0.5,   [1,2] ),
    ( 0.5,   [3,4] ),
    (-0.5,   [2,3] ),
    ( 1.0,   [4,5] ),
    ( 1.0,   [1,2] ),
    (-0.5,   [3,4] ),
    (-0.5,   [2,3] ),
    (-0.5,   [4,5] ),
    ( 1.0,   [3,4] ),
    ( 0.5,   [2,3] ),
    ( 1-p2,  [4,5] ),
    # ( -p1,   [3,4] ),
    ( 1-p1,  [3,4] ),
]

# ------------------------------------------------------------
# 3.  Utility: map (i,j) → pair_id in NEIGHBORS (order-agnostic)
# ------------------------------------------------------------
def pair_to_id(i: int, j: int) -> int:
    """Return k such that set(NEIGHBORS[k]) == {i,j}."""
    a, b = sorted((i, j))                       # unordered → ordered
    # Boolean mask over rows of NEIGHBORS
    mask = jnp.all(NEIGHBORS == jnp.array([a, b]), axis=1)
    # indices where mask==True   (at most one for a proper table)
    idx_arr = jnp.nonzero(mask, size=1, fill_value=-1)[0]
    idx     = int(idx_arr.item())               # ← convert scalar array → int
    if idx == -1:
        raise ValueError(f"Pair {(i, j)} not present in NEIGHBORS.")
    return idx


# ------------------------------------------------------------
# 4.  Build the action list in env format
# ------------------------------------------------------------
actions = [
    ( pair_to_id(i, j), jnp.array([p], dtype=jnp.float32) )
    for p, (i, j) in gate_specs
]

# ------------------------------------------------------------
# 5.  Run one deterministic episode
# ------------------------------------------------------------
env   = ExchangeCNOTEnv(EnvParams(max_depth=len(actions)))
key   = jax.random.PRNGKey(42)
state, obs = env.reset(key)

print("=====  Fong–Wandzura 22-pulse unit test  =====")
for t, act in enumerate(actions, start=1):
    key, sub = jax.random.split(key)
    state, obs, r, done, _ = env.step(sub, state, act)

    pair_id = act[0]
    p_val   = act[1].item()       # ← scalar
    reward  = r.item()            # ← scalar
    fid64   = state.fid64.item()  # ← scalar

    print(f"step {t:02d}  pair_id={pair_id}  p={p_val: .6f}  "
          f"reward={reward: .3f}  fid64={fid64:.6f}")
    if done:
        print(f"*** episode terminated after {t} steps ***")
        break


# ------------------------------------------------------------
# 6.  Post-mortem: check full-matrix fidelity & block fidelity
# ------------------------------------------------------------
phase  = jnp.vdot(state.U, TARGET_FULL)
phase /= jnp.abs(phase)                    # remove global phase
fid64  = _fidelity(state.U*phase.conj(), TARGET_FULL)
fid9   = _fidelity(_block(state.U), _block(TARGET_FULL))

print("\nFinal fidelities:")
print(f"  ‖U_final − U_target‖_F  (64×64)  = {1 - float(fid64):.2e}")
print(f"  DFS-block fidelity (9×9)          = {float(fid9):.6f}")

assert jnp.isclose(fid64, 1.0, atol=1e-6), "Full 64×64 fidelity < 0.999999"
assert jnp.isclose(fid9 , 1.0, atol=1e-6), "Logical-block fidelity < 0.999999"
print("✅  FW sequence reproduces the target gate exactly.")


=====  Fong–Wandzura 22-pulse unit test  =====
step 01  pair_id=3  p= 1.695913  reward=-1.000  fid64=0.229334
step 02  pair_id=4  p= 0.108173  reward=-1.000  fid64=0.223945
step 03  pair_id=2  p= 0.500000  reward=-1.000  fid64=0.177044
step 04  pair_id=3  p= 1.000000  reward=-1.000  fid64=0.110696
step 05  pair_id=2  p=-0.500000  reward= 1.500  fid64=0.114028
step 06  pair_id=4  p=-0.500000  reward= 1.000  fid64=0.127438
step 07  pair_id=1  p= 1.000000  reward= 8.500  fid64=0.290135
step 08  pair_id=3  p=-0.500000  reward= 1.000  fid64=0.291383
step 09  pair_id=2  p=-0.500000  reward=-1.000  fid64=0.263663
step 10  pair_id=4  p= 1.000000  reward= 6.500  fid64=0.334522
step 11  pair_id=1  p=-0.500000  reward=-5.750  fid64=0.221102
step 12  pair_id=3  p= 0.500000  reward=-1.500  fid64=0.275630
step 13  pair_id=2  p=-0.500000  reward=-4.250  fid64=0.263796
step 14  pair_id=4  p= 1.000000  reward=-4.500  fid64=0.224025
step 15  pair_id=1  p= 1.000000  reward= 15.750  fid64=0.448051
step 16

In [1]:
# ─────────────────────────────────────────────────────────────────────────────
# train_exchange_cnot.py
# Search for shorter exchange–only CNOT circuits with PPO (purejaxrl style)
# ─────────────────────────────────────────────────────────────────────────────
import jax, jax.numpy as jnp
from functools import partial

# --- local modules you already uploaded -------------------------------------
from exchange_cnot_env import ExchangeCNOTEnv, EnvParams         # the env
from ppo_qcrl import ActorCritic                                 # model
from purejaxrl.purejaxrl.wrappers import LogWrapper, FlattenObservationWrapper       # purejaxrl

import optax, distrax, numpy as np
from flax.training.train_state import TrainState
from flax.linen.initializers import constant, orthogonal
import gymnax                                                     # just for spaces helpers

# ----------------------------------------------------------------------------
# 0.  Config
# ----------------------------------------------------------------------------
config = dict(
    TOTAL_TIMESTEPS = 1_000_000,     # feel free to raise – keeps compilation tolerable
    NUM_ENVS        = 16,            # vectorised environments
    NUM_STEPS       = 128,           # rollout length
    UPDATE_EPOCHS   = 4,
    NUM_MINIBATCHES = 8,
    LR              = 3e-4,
    ACTOR_ACTIVATION = "tanh",       # "relu" also ok
    GAMMA           = 0.995,
    GAE_LAMBDA      = 0.95,
    CLIP_EPS        = 0.2,
    ENT_COEF        = 1e-2,
    VF_COEF         = 0.5,
    MAX_GRAD_NORM   = 0.5,
    ANNEAL_LR       = True,
    DEBUG           = True,          # prints episodic return every few seconds
)

# ----------------------------------------------------------------------------
# 1.  Environment (with wrappers) & spaces helpers
# ----------------------------------------------------------------------------
base_env      = ExchangeCNOTEnv(EnvParams(max_depth=18, obs_mode="block"))
env_params    = base_env.default_params
env           = FlattenObservationWrapper(base_env)           # obs → 1-D
env           = LogWrapper(env)                               # episodic stats

disc_dim = env.action_space(env_params).spaces[0].n           # == 5 neighbour pairs
obs_shape = env.observation_space(env_params).shape

# ----------------------------------------------------------------------------
# 2.  Policy / value network
# ----------------------------------------------------------------------------
class AC(ActorCritic):
    disc_dim: int = disc_dim
    activation: str = config["ACTOR_ACTIVATION"]

network = AC()

# ----------------------------------------------------------------------------
# 3.  Training utilities
# ----------------------------------------------------------------------------
def linear_schedule(count, base_lr=config["LR"], total_updates=None):
    """
    Linear LR decay identical to Stable-Baselines3.
    """
    frac = 1.0 - count / total_updates if total_updates else 1.0
    return base_lr * frac

def make_optimizer(total_updates):
    if config["ANNEAL_LR"]:
        sched = partial(linear_schedule, total_updates=total_updates)
        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.adam(learning_rate=sched, eps=1e-5)
        )
    else:
        tx = optax.chain(
            optax.clip_by_global_norm(config["MAX_GRAD_NORM"]),
            optax.adam(config["LR"], eps=1e-5)
        )
    return tx

# ----------------------------------------------------------------------------
# 4.  Main training function (pure-JAX)
# ----------------------------------------------------------------------------
def make_ppo_train(config):
    total_updates   = (config["TOTAL_TIMESTEPS"] //
                       config["NUM_STEPS"] //
                       config["NUM_ENVS"])
    minibatch_size  = (config["NUM_ENVS"] * config["NUM_STEPS"] //
                       config["NUM_MINIBATCHES"])
    tx              = make_optimizer(total_updates)

    def train(rng):
        # -- network init -----------------------------------------------------
        rng, net_key = jax.random.split(rng)
        params  = network.init(net_key, jnp.zeros(obs_shape))
        state   = TrainState.create(apply_fn=network.apply, params=params, tx=tx)

        # -- env init ---------------------------------------------------------
        rng, env_key = jax.random.split(rng)
        keys   = jax.random.split(env_key, config["NUM_ENVS"])
        obs,   env_state = jax.vmap(env.reset, in_axes=(0,None))(keys, env_params)

        # --------------------------------------------------------------------
        # vmap helpers for multi-env rollout
        # --------------------------------------------------------------------
        Transition = gymnax.collector.Transition  # tiny struct from purejaxrl >=0.3
        def policy_step(state, obs, rng):
            """
            One policy inference step – returns action, value, log-prob.
            """
            rng, sample_key = jax.random.split(rng)
            pi, value = network.apply(state.params, obs)
            action    = pi.sample(seed=sample_key)
            logp      = pi.log_prob(action)
            return rng, action, value, logp

        def env_step(rng, env_state, action):
            rng, step_key = jax.random.split(rng)
            new_state, new_obs, reward, done, info = env.step(
                step_key, env_state, action, env_params
            )
            return rng, new_state, new_obs, reward, done, info

        # --------------------------------------------------------------------
        # Main loop (jax.lax.scan over updates)
        # --------------------------------------------------------------------
        def update_loop(carry, unused_t):
            train_state, env_state, last_obs, rng = carry
            # ===== COLLECT TRAJECTORY =====
            def rollout_step(carry, t):
                rng, env_state, obs = carry
                rng, act, val, logp = jax.vmap(policy_step, in_axes=(None,0,0))(
                    train_state, obs, jax.random.split(rng, config["NUM_ENVS"])
                )
                rng, env_state, next_obs, rew, done, info = jax.vmap(env_step, in_axes=(0,0,0))(
                    jax.random.split(rng, config["NUM_ENVS"]), env_state, act
                )
                trans = Transition(obs, act, rew, done, val, logp)
                return (rng, env_state, next_obs), trans

            (rng, env_state, next_obs), traj = jax.lax.scan(
                rollout_step,
                (rng, env_state, obs),
                None,
                length=config["NUM_STEPS"]
            )

            # ===== GAE / target value =====
            _, last_val = jax.vmap(policy_step, in_axes=(None,0,0))(
                train_state, next_obs,
                jax.random.split(rng, config["NUM_ENVS"])
            )

            def _gae(carry, transition):
                gae, next_val = carry
                delta = transition.reward + config["GAMMA"] * next_val * (1.0 - transition.done) - transition.value
                gae   = delta + config["GAMMA"] * config["GAE_LAMBDA"] * (1.0 - transition.done) * gae
                return (gae, transition.value), gae

            (_, _), advantages = jax.lax.scan(
                _gae, (jnp.zeros_like(last_val), last_val), traj, reverse=True
            )
            targets = advantages + traj.value

            # ===== SHUFFLE / MINIBATCH SGD =====
            batch_size = config["NUM_ENVS"] * config["NUM_STEPS"]
            flat = gymnax.collector.flatten_batch((traj, advantages, targets))
            permutation = jax.random.permutation(rng, batch_size)
            shuffled = jax.tree_util.tree_map(lambda x: jnp.take(x, permutation, axis=0), flat)

            def sgd_epoch(state, unused_e):
                def minibatch_update(state, mb_idx):
                    mb = jax.tree_util.tree_map(
                        lambda x: jnp.take(x, mb_idx, axis=0), shuffled
                    )

                    def loss_fn(params):
                        pi, values = network.apply(params, mb.obs)
                        logp = pi.log_prob(mb.action)
                        # actor loss
                        ratio = jnp.exp(logp - mb.log_prob)
                        adv   = (mb.advantages - mb.advantages.mean()) / (mb.advantages.std() + 1e-8)
                        loss_actor = -jnp.mean(jnp.minimum(
                            ratio * adv,
                            jnp.clip(ratio, 1.0 - config["CLIP_EPS"], 1.0 + config["CLIP_EPS"]) * adv
                        ))
                        # critic loss
                        v_pred_clipped = mb.value + (values - mb.value).clip(-config["CLIP_EPS"], config["CLIP_EPS"])
                        v_loss = 0.5 * jnp.mean(jnp.maximum(
                            (values - mb.targets)**2,
                            (v_pred_clipped - mb.targets)**2
                        ))
                        # entropy bonus
                        entropy = jnp.mean(pi.entropy())
                        loss = loss_actor + config["VF_COEF"] * v_loss - config["ENT_COEF"] * entropy
                        return loss, (loss_actor, v_loss, entropy)

                    grads_fn = jax.value_and_grad(loss_fn, has_aux=True)
                    (loss, aux), grads = grads_fn(state.params)
                    state = state.apply_gradients(grads=grads)
                    return state, aux

                # build minibatch indices
                idx = jnp.arange(batch_size).reshape(config["NUM_MINIBATCHES"], -1)
                state, metrics = jax.lax.scan(minibatch_update, state, idx)
                return state, metrics

            train_state, _ = jax.lax.scan(sgd_epoch, train_state, None, length=config["UPDATE_EPOCHS"])

            # OPTIONAL: simple callback
            if config["DEBUG"]:
                def _print(info):
                    # log reward of *finished* episodes only
                    returns = info["returned_episode_returns"][info["returned_episode"]]
                    steps   = info["timestep"][info["returned_episode"]] * config["NUM_ENVS"]
                    for r, s in zip(returns.tolist(), steps.tolist()):
                        print(f"Step {s:>7} | episodic return: {r:8.3f}")
                jax.debug.callback(_print, traj.info)

            carry_out = (train_state, env_state, next_obs, rng)
            return carry_out, {}

        # ---- run -----------------------------------------------------------
        carry = (state, env_state, obs, rng)
        carry, _ = jax.lax.scan(update_loop, carry, None, length=total_updates)
        return carry[0]   # trained TrainState

    # jit once outside of the call site
    return jax.jit(train, donate_argnums=(0,))

# ----------------------------------------------------------------------------
# 5.  Launch
# ----------------------------------------------------------------------------
rng = jax.random.PRNGKey(0)
train_fn = make_ppo_train(config)
final_state = train_fn(rng)

print("\nTraining finished – parameters in `final_state.params`")


TypeError: 'Tuple' object is not callable